In [7]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, random_split
from fastai.data.core import DataLoaders
import torch.nn as nn
from fastai.learner import Learner
from fastai.metrics import accuracy
from fastai.losses import CrossEntropyLossFlat
from torch.utils.data import TensorDataset, random_split, DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn as nn
from fastai.learner import Learner
from fastai.losses import CrossEntropyLossFlat
from fastai.metrics import accuracy
import random
from fastai.callback.tracker import EarlyStoppingCallback


torch.manual_seed(42)
np.random.seed(42)
random.seed(42)



df = yf.download("^GSPC", start="2000-01-01")

/tmp/ipython-input-2668286915.py:32: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("^GSPC", start="2000-01-01")
[*********************100%***********************]  1 of 1 completed


In [8]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2000-01-03,1455.219971,1478.000000,1438.359985,1469.250000,931800000
2000-01-04,1399.420044,1455.219971,1397.430054,1455.219971,1009000000
2000-01-05,1402.109985,1413.270020,1377.680054,1399.420044,1085500000
2000-01-06,1403.449951,1411.900024,1392.099976,1402.109985,1092300000
2000-01-07,1441.469971,1441.469971,1400.729980,1403.449951,1225200000


Elimine un nivel del índice de columnas generado por yfinance

In [9]:
df.columns = df.columns.droplevel(1)


Obtenemos el retorno logarítmico del día usando el cierre de hoy dividido entre el de ayer. Quito la primera fila porque no puede calcularse su retorno (queda como NaN)

In [10]:
df['log_ret'] = np.log(df['Close'] / df['Close'].shift(1))

df = df.dropna(subset=['log_ret'])

In [11]:
df.head()

Price,Close,High,Low,Open,Volume,log_ret
Date,,,,,,
2000-01-04,1399.420044,1455.219971,1397.430054,1455.219971,1009000000,-0.039099
2000-01-05,1402.109985,1413.270020,1377.680054,1399.420044,1085500000,0.001920
2000-01-06,1403.449951,1411.900024,1392.099976,1402.109985,1092300000,0.000955
2000-01-07,1441.469971,1441.469971,1400.729980,1403.449951,1225200000,0.026730
2000-01-10,1457.599976,1464.359985,1441.469971,1441.469971,1064800000,0.011128


Agregamos label

Aqui calculo la volatilidad futura. Es la desviación estandar de los próximos 10 días.

Don shift(-10) muevo ese valor hacia arriba para que corresponda al día donde quiero predecir

Para esta etapa recurrí a asistencia técnica de ChatGPT con el fin de validar que la ventana de días futuros estuviera correctamente alineada con el instante temporal t. Esta comprobación era esencial, ya que cualquier desalineación en la volatilidad futura afectaría directamente la generación de etiquetas y, por lo tanto, el desempeño del modelo.

In [12]:
window_vol = 10

# Volatilidad futura, std de los próximos N días
df['rv_future'] = df['log_ret'].rolling(window_vol).std().shift(-window_vol)

print(df['rv_future'])

Date
2000-01-04    0.011474
2000-01-05    0.011996
2000-01-06    0.012133
2000-01-07    0.012418
2000-01-10    0.011876
                ...   
2025-11-26         NaN
2025-11-28         NaN
2025-12-01         NaN
2025-12-02         NaN
2025-12-03         NaN
Name: rv_future, Length: 6519, dtype: float64


Calculo el umbral

In [13]:
threshold = df['rv_future'].median()
print(threshold)

0.008325207617525083


Generación del label y limpieza de NaN


In [14]:
df['label'] = (df['rv_future'] > threshold).astype(int)

df = df.dropna(subset=['rv_future'])

Total de 0 y 1

In [15]:
df['label'].value_counts()

,count
label,
0,3255
1,3254


In [16]:
df.head()

Price,Close,High,Low,Open,Volume,log_ret,rv_future,label
Date,,,,,,,,
2000-01-04,1399.420044,1455.219971,1397.430054,1455.219971,1009000000,-0.039099,0.011474,1
2000-01-05,1402.109985,1413.270020,1377.680054,1399.420044,1085500000,0.001920,0.011996,1
2000-01-06,1403.449951,1411.900024,1392.099976,1402.109985,1092300000,0.000955,0.012133,1
2000-01-07,1441.469971,1441.469971,1400.729980,1403.449951,1225200000,0.026730,0.012418,1
2000-01-10,1457.599976,1464.359985,1441.469971,1441.469971,1064800000,0.011128,0.011876,1


In [17]:
df.describe()

Price,Close,High,Low,Open,Volume,log_ret,rv_future,label
count,6509.000000,6509.000000,6509.000000,6509.000000,6.509000e+03,6509.000000,6509.000000,6509.000000
mean,2248.814759,2261.320538,2234.620184,2248.478699,3.413252e+09,0.000233,0.010126,0.499923
std,1425.332964,1431.619479,1417.896291,1425.062347,1.512048e+09,0.012234,0.007124,0.500038
min,676.530029,695.270020,666.789978,679.280029,0.000000e+00,-0.127652,0.001459,0.000000
25%,1209.569946,1215.689941,1200.739990,1209.469971,2.301860e+09,-0.004757,0.005767,0.000000
50%,1520.770020,1529.390015,1514.140015,1520.770020,3.525160e+09,0.000634,0.008325,0.000000
75%,2878.199951,2891.110107,2860.709961,2876.090088,4.252100e+09,0.005872,0.012162,1.000000
max,6890.890137,6920.339844,6870.729980,6910.950195,1.145623e+10,0.109572,0.075383,1.000000
